## Đồ án cuối kỳ môn Nhập môn Khoa học dữ liệu nhóm 40 gồm 2 thành viên:
- Phạm Như Quyền - 1712714
- Nguyễn Hữu Thắng - 1712756

## GIỚI THIỆU ĐỀ TÀI CUỐI KỲ:
IMDb (Internet movie database) là một cơ sở dữ liệu trực tuyến chứa các thông tin liên quan đến các bộ phim, chương trình truyền hình, trò chơi điện tử và các nội dung được phát trực tuyến khác.

Đồ án này dựa trên các thông tin liên quan đến các bộ phim như điểm đánh giá, nội dung tóm tắt, năm sản xuất đã thu thập được trên IMDb để bắt đầu tìm hiểu nghiên cứu về các bộ phim cũng như đưa ra một mô hình để thực hiện dự đoán ratings các bộ phim thông qua 1 số yếu tố của bộ phim đó.

Đồ án được thực hiện gồm 3 bước chính:

Bước 1 : Thu thập dữ liệu sẽ được thực hiện trên tệp notebook "40_CrawlingDataIMDB.ipynb", sau khi thu thập, dữ liệu sẽ được lưu vào tệp "movie_IMDB.csv"

Bước 2: Khám phá và tiền xử lý dữ liệu.

Bước 3: Xây dựng mô hình

Bước 2,3 sẽ được thực hiện trong file "Explore_Preprocessing_BuildModel_IMDbData.ipynb"

Nguồn thông tin : https://en.wikipedia.org/wiki/IMDb

## IMPORT THƯ VIỆN PYTHON CẦN THIẾT

In [13]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### THU THẬP DỮ LIỆU TỪ TRANG WEB IMDB BẰNG CÁCH PARSE HTML
Nguồn dữ liệu : https://www.imdb.com/search/title/?groups=top_1000&count=100&start=1&ref_=adv_nxt

In [14]:
page = 1
movie_df = pd.DataFrame()
while True:
    url = 'https://www.imdb.com/search/title/?groups=top_1000&count=100&start='+ str(page)+'&ref_=adv_nxt'
    data_page = requests.get(url,headers={"Accept-Language": "en-US"})
    print(url)
    titles = [] # List chứa các tiêu đề
    certificates = [] # List chứa các chứng chỉ của phim
    ratings = [] # List chứa điểm đánh giá của phim
    genres = [] # List chứa các thể loại của phim
    votes =[] # List chứa số lượng vote cho phim
    metascores = [] # List chứa điểm đánh giá trung bình của phim từ 0 - 100
    runtime_list = [] # List chứa thời lượng phim
    directors_list = [] # List chứa các đạo diễn của phim
    stars_list = [] # List chứa các ngôi sao của phim
    grosses_list =[] # List chứa doanh thu của phim
    introduction_list = [] # List chứa giới thiệu nội dung của phim
    produce_year_list = [] # List chứa năm sản xuất của phim
    soup = BeautifulSoup(data_page.text)
    list_item_content = soup.findAll('div',class_ = 'lister-item-content')
    print(len(list_item_content))
    for movie_content in list_item_content:
        list_muted_text = movie_content.findAll('p',class_='text-muted')
        introduction_list.append(list_muted_text[1].text.replace("\n ",""))
        item_header = movie_content.find('h3',class_='lister-item-header')
        title = item_header.find('a')
        rating = movie_content.find('strong')
        certificate = movie_content.find('span',class_ = 'certificate')
        vote = movie_content.find('p',class_ ='sort-num_votes-visible')
        genre = movie_content.find('span',class_='genre')
        metascore = movie_content.find('span',class_='metascore favorable')
        runtime = movie_content.find('span',class_='runtime')
        p_content = movie_content.find('p',class_='')
        produce_year = movie_content.find('span',class_='lister-item-year')
        produce_year = produce_year.text.replace("(","")
        produce_year = produce_year.replace(")","")
        produce_year_list.append(produce_year)
        if p_content != None:
            p_content_split = p_content.text.split('|')
            directors = p_content_split[0].replace("\n","")
            directors = directors.replace("Director:","")
            directors = directors.replace("Directors:","")
            stars = p_content_split[1].replace("\n","")
            stars = stars.replace("Stars:","")
            stars_list.append(stars)
            directors_list.append(directors)
        else:
            directors_list.append(None)
            stars_list.append(None)
        if certificate != None:
            certificates.append(certificate.text.strip())
        else:
            certificates.append(certificate)
        if "|" in vote.text:
            vote_gross = vote.text.split("|")
            vote_value = vote_gross[0]
            vote_value = vote_value.replace("\n","")
            vote_value = vote_value.replace(",","")
            vote_value = vote_value.replace("Votes:","")
            gross_value = vote_gross[1]
            gross_value = gross_value.replace("\n","")
            gross_value = gross_value.replace("Gross:","")
            gross_value = gross_value.replace("$","")
            gross_value = gross_value.replace("M","")
            gross_value = gross_value.replace(" ","")
            votes.append(vote_value)
            grosses_list.append(gross_value)
        else:
            vote_value = vote.text.replace("\n","")
            vote_value = vote_value.replace("Votes:","")
            vote_value = vote_value.replace(",","")
            votes.append(vote_value)
            grosses_list.append(None)
        if rating != None:
            ratings.append(rating.text.strip())
        else:
            ratings.append(rating)
        if title != None:
            titles.append(title.text.strip())
        else:
            titles.append(title)
        if genre != None:
            genres_text = genre.text.replace(" ","")
            genres_text = genres_text.strip()
            genres.append(genres_text)
        else:
            genres.append(genre)
        if metascore != None:
            metascores.append(metascore.text.strip())
        else:
            metascores.append(metascore)
        if runtime != None:
            runtime_text = runtime.text.strip()
            runtime_text = runtime_text.replace(" min","")
            runtime_list.append(runtime_text)
        else:
            runtime_list.append(runtime)

    df = pd.DataFrame({
                       'titles':titles,
                       'genres':genres,
                       'certificates':certificates,
                       'votes':votes,
                       'metascores':metascores,
                       'ratings':ratings,
                       'directors':directors_list,
                       'stars':stars_list,
                       "gross":grosses_list,
                      'introduction':introduction_list,
                      'produce_year':produce_year_list,
                      'runtime':runtime_list,
                       'ratings':ratings})
    movie_df = movie_df.append(df)
    page = page + 100
    if page > 1000:
        break

https://www.imdb.com/search/title/?groups=top_1000&count=100&start=1&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=101&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=201&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=301&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=401&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=501&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=601&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=701&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=801&ref_=adv_nxt
100
https://www.imdb.com/search/title/?groups=top_1000&count=100&start=901&ref_=adv_nxt
100


In [15]:
movie_df.head()

,titles,genres,certificates,votes,metascores,ratings,directors,stars,gross,introduction,produce_year,runtime
0,Soul,"Animation,Adventure,Comedy",PG,126286,83,8.1,"Pete Docter, Kemp Powers","Jamie Foxx, Tina Fey, Graham Norton, Rach...",None,A musician who has lost his passion for mus...,2020,100
1,Another Round,"Comedy,Drama",Not Rated,26017,81,7.8,Thomas Vinterberg,"Mads Mikkelsen, Thomas Bo Larsen, Magnus ...",None,"Four friends, all high school teachers, tes...",2020,117
2,Avengers: Endgame,"Action,Adventure,Drama",PG-13,800050,78,8.4,"Anthony Russo, Joe Russo","Robert Downey Jr., Chris Evans, Mark Ruff...",858.37,After the devastating events of Avengers: I...,2019,181
3,The Gentlemen,"Action,Comedy,Crime",R,230044,None,7.8,Guy Ritchie,"Matthew McConaughey, Charlie Hunnam, Mich...",None,An American expat tries to sell off his hig...,2019,113
4,Parasite,"Comedy,Drama,Thriller",R,540872,96,8.6,Bong Joon Ho,"Song Kang-Ho, Lee Sun-kyun, Cho Yeo-jeong...",53.37,Greed and class discrimination threaten the...,2019,132


In [16]:
movie_df.to_csv("movie_IMDB.csv",encoding="utf-8",index=False)